# Example notebook

Displays how the EM module works on an RIR stored in example.mat in matlab format.

In [24]:
from __future__ import print_function
import numpy as np
from IPython.display import clear_output

from scipy.signal import lfilter, stft
from scipy.linalg import toeplitz
from scipy.signal import convolve2d, correlate2d
import time
import scipy.io

if sys.version_info.major == 3:
    import _pickle as pickle
else:
    import cPickle as pickle
    
import EM

In [25]:
# Definition of the progress bar function for better readability
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### Load example RIR
Load an exemple dictionnary containing a RIR and the corresponding true parameters with pickle.

In [26]:
filename = "examples/absorption/abs7_seed0.pkl"

In [27]:
try:
    with open(filename, 'rb') as in_file:
        dic = pickle.load(in_file)
except UnicodeDecodeError:
    with open(filename, 'rb') as in_file:
        dic = pickle.load(in_file, encoding="latin1")

In [32]:
h = dic["h"] # Example RIR

### Initialize EM algorithm

In [34]:
algo = EM.EM(h)

Initializing EM algorithm
la init = 5.645029269476762e-06

Updating R

Updating mu
Log-probability difference = -68308.06358093786


### Iterate 50 times the expectation and maximization steps

In [36]:
n_iter = 50

for i in range(n_iter):
    algo.iteration()
    update_progress(i / n_iter)

update_progress(1)

Progress: [####################] 100.0%
